In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
import time
import gc

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B").to('cuda')
question="hi how are you"
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
# prompt=tok.apply_chat_template(converstation)
prompt=tok(question,return_tensors='pt').to('cuda')
out=model.generate(**prompt)


In [3]:
out[0]

tensor([128000,   6151,   1268,    527,    499,     30,    602,    574,  20910,
           422,    499,   1436,   1520,    757,    449,   2555,     11,    602,
           617,    264,    220,   2550,     22,    220,     19],
       device='cuda:0')

In [4]:
# If out is a tensor (e.g., shape [1, seq_len])
decoded = tok.decode(out[0], skip_special_tokens=True)
print(decoded)


hi how are you? i was wondering if you could help me with something, i have a 1997 4


In [2]:
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
print(model)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [3]:
model.model.rotary_emb

LlamaRotaryEmbedding()

In [3]:
for i in model.model.layers:
    print(i)
    break

LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear(in_features=2048, out_features=512, bias=False)
    (v_proj): Linear(in_features=2048, out_features=512, bias=False)
    (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)


In [4]:
model=model.to('cpu')

In [5]:
torch.cuda.empty_cache()

In [8]:
question=input()
converstation=[
    {
        "role":"system",
        "content":"You are an assitant help user by solving their query"
    },
    {
        "role":"user",
        "content":f"query-{question}"
    }
]

In [35]:
question="hi how are "
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
# prompt=tok.apply_chat_template(converstation)
prompt=tok(question,return_tensors='pt').to('cuda')
# prompt.to('cpu'//)


In [36]:
prompt

{'input_ids': tensor([[128000,   6151,   1268,    527,    220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

In [37]:
# out=model.generate(**prompt,max_new_tokens=1000)
# print(tok.decode(out[0]))

In [38]:
input_ids=prompt['input_ids']
del prompt
torch.cuda.empty_cache()

In [39]:
input_ids.device

device(type='cuda', index=0)

In [ ]:
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
# del input_ids
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()


In [20]:
# model.model.decoder.embed_positions.to('cuda')
# x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
# model.model.decoder.embed_positions.to('cpu')
# # del model.model.decoder.embed_positions
# torch.cuda.empty_cache()

In [41]:
print(f"Model's max_position_embeddings: {model.config.max_position_embeddings}")

Model's max_position_embeddings: 131072


In [42]:
x.shape

torch.Size([1, 5, 2048])

In [43]:
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)
cnt=0
for i in model.model.layers:
    cnt+=1
    print(cnt)
    i.to('cuda')
    x.to('cuda')
    # print(x)
    with torch.no_grad():
        x=i(x,position_embeddings=(cos,sin))
        # print(x)
    i.to('cpu')
    torch.cuda.empty_cache()
    x=x[0]
    # x.to('cpu')
    torch.cuda.empty_cache()
        # time.sleep(3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [44]:
x.shape

torch.Size([1, 5, 2048])

In [45]:
x

tensor([[[ 4.7762, -5.0166, 13.8534,  ..., -1.9152,  1.8989, -3.0755],
         [-0.2314,  1.3026,  1.7797,  ..., -0.2708, -1.0364, -0.2083],
         [ 0.2276,  0.8936,  0.8521,  ..., -0.0246, -1.4944, -0.0458],
         [ 0.0748,  1.2204,  0.5945,  ..., -0.7305, -1.4607, -0.6370],
         [ 0.6602,  1.3918, -0.1689,  ..., -1.1154, -1.7489, -0.1802]]],
       device='cuda:0')

In [46]:
x[:,-1,:]

tensor([[ 0.6602,  1.3918, -0.1689,  ..., -1.1154, -1.7489, -0.1802]],
       device='cuda:0')

In [47]:
with torch.no_grad():
    model.model.norm.to('cuda')
    x=model.model.norm(x)
    model.model.norm.to('cpu')
    torch.cuda.empty_cache()
    model.lm_head.to('cuda')
    x=model.lm_head(x)
    model.lm_head.to('cpu')
    torch.cuda.empty_cache()

In [48]:
x1.shape

AttributeError: 'list' object has no attribute 'shape'

In [57]:
x

tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4011,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       device='cuda:0')

In [60]:
prompt

NameError: name 'prompt' is not defined

In [56]:
x1=x[:,-1,:]
x1=torch.argmax(x1,dim=-1,keepdim=True)
# print(tok.decode(x1[0]),end=" ")
print(x1)

tensor([[17]], device='cuda:0')


In [ ]:
def model_inference(question,tokens=250):
    for i in range(tokens):
        prompt=tok(question,return_tensors='pt').to('cpu')
        input_ids=prompt['input_ids'].to('cuda')
        attention_mask=prompt['attention_mask']
        del prompt
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        input_ids.to('cpu')
        del input_ids
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cpu')
        torch.cuda.empty_cache()
        # Safe check
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        cnt=0
        for i in model.model.layers:
            cnt+=1
            # print(cnt)
            i.to('cuda')
            x.to('cuda')
            # print(x)
            with torch.no_grad():
                x=i(x,position_embeddings=(cos,sin))
                # print(x)
            i.to('cpu')
            torch.cuda.empty_cache()
            x=x[0]
            # x.to('cpu')
            torch.cuda.empty_cache()
            
        with torch.no_grad():
            model.model.norm.to('cuda')
            x=model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()
            model.lm_head.to('cuda')
            x=model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()
            
        x1=x.argmax(-1)
        print(tok.decode(x1[0]),end="\n")
        question=question+" "+tok.decode(x1[0])
        

In [9]:
def model_inference(prompt_text, tokens=250):
    # Initial tokenization
    prompt = tok(prompt_text, return_tensors='pt')
    input_ids = prompt['input_ids'].to('cuda')  # [1, seq_len]
    del prompt
    torch.cuda.empty_cache()

    for _ in range(tokens):
        # Run only the latest token through embed layer and model
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        model.model.embed_tokens.to('cpu')
        torch.cuda.empty_cache()

        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)

        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x, position_ids=position_ids)

        # Transformer blocks
        for layer in model.model.layers:
            layer.to('cuda')
            x = x.to('cuda')
            with torch.no_grad():
                x = layer(x, position_embeddings=(cos, sin))
            layer.to('cpu')
            torch.cuda.empty_cache()
            x = x[0]  # Remove tuple

        # Final norm and lm_head
        with torch.no_grad():
            model.model.norm.to('cuda')
            x = model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()

            model.lm_head.to('cuda')
            logits = model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()

        next_token_logits = logits[:, -1, :]  # Only the last token matters
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)  # shape [1,1]

        # Append predicted token
        input_ids = torch.cat([input_ids, next_token], dim=-1)

        # Decode and print latest token
        print(tok.decode(next_token[0]), end="", flush=True)


In [11]:
model_inference("Hi how are you ?")

 I am very happy to meet you all. I am a very friendly person and I love to meet new people. I am a very good listener and I am very easy going. I am a very good cook and I love to cook. I am a very good dancer and I love to dance. I am a very good singer and I love to sing. I am a very good swimmer and I love to swim. I am a very good gardener and I love to garden. I am a very good cook and I love to cook. I am a very good dancer and I love to dance. I am a very good singer and I love to sing. I am a very good swimmer and I love to swim. I am a very good gardener and I love to garden. I am a very good cook and I love to cook. I am a very good dancer and I love to dance. I

KeyboardInterrupt: 

In [52]:
def tokenize_with_oov(text, tokenizer, max_token_id=2048, oov_token='[OOV]'):
    # First, add [OOV] token to tokenizer if not present
    if oov_token not in tokenizer.get_vocab():
        tokenizer.add_special_tokens({'additional_special_tokens': [oov_token]})

    oov_id = tokenizer.convert_tokens_to_ids(oov_token)

    # Raw tokenization
    encoding = tokenizer(text, return_tensors='pt', add_special_tokens=False)
    input_ids = encoding['input_ids'][0]

    # Replace tokens ≥ max_token_id with OOV
    processed_ids = []
    for token_id in input_ids:
        if token_id >= max_token_id:
            processed_ids.append(oov_id)
        else:
            processed_ids.append(token_id)

    return torch.tensor([processed_ids]), encoding['attention_mask']

def model_inference(question, tokens=250):
    try:
        for _ in range(tokens):
            # prompt = tok(question, return_tensors='pt').to('cpu')
            # input_ids = prompt['input_ids']
            # attention_mask = prompt['attention_mask']
            input_ids, attention_mask = tokenize_with_oov(question, tok)

            del prompt
            torch.cuda.empty_cache()

            # Truncate if too long
            batch_size, seq_len = input_ids.shape
            max_positions = model.config.max_position_embeddings
            if seq_len > max_positions:
                input_ids = input_ids[:, -max_positions:]
                attention_mask = attention_mask[:, -max_positions:]
                seq_len = input_ids.shape[1]

            # Embeddings
            model.model.decoder.embed_tokens.to('cpu')
            x = model.model.decoder.embed_tokens(input_ids)

            # Positional IDs
            position_ids = torch.arange(seq_len, dtype=torch.long, device='cpu')
            position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)

            model.model.decoder.embed_positions.to('cpu')
            x = x + model.model.decoder.embed_positions(position_ids)

            # Decoder layers
            for layer in model.model.decoder.layers:
                layer.to('cpu')
                x = x.to('cpu')
                with torch.no_grad():
                    x = layer(x, attention_mask=attention_mask.to(torch.float32))[0]
                layer.to('cpu')
                x = x.to('cpu')
                torch.cuda.empty_cache()

            # Final LM head
            model.lm_head.to('cpu')
            logits = model.lm_head(x.to('cpu'))
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()

            # Decode
            next_token_id = logits.argmax(-1)
            next_token_text = tok.decode(next_token_id[0], skip_special_tokens=True)
            print(next_token_text, end=" ")
            question += " " + next_token_text

    except Exception as e:
        print("\n[Exception Caught]")
        print("Question So Far:", question)
        prompt = tok(question, return_tensors='pt').to('cpu')
        input_ids = prompt['input_ids']
        x = model.model.decoder.embed_tokens(input_ids)
        y = torch.arange(x.shape[1]).unsqueeze(0)
        print("Embedding Shape:", x[0].shape)
        print("Position ID Shape:", y.shape)
        print("Error:", e)


In [56]:
model_inference("how are you 're post")

.''. you you'rewepost navigation .''. your you'reTheze is It.''..''.'re'reiresolicy姫conservancy how are you 're post .''. you you'rewepost navigation .''. your you'reTheze is It.''..''.'re'reiresolicy姫conservancy

torch.Size([35, 768])
torch.Size([1, 35])
index out of range in self


In [15]:
import torch
import gc

def model_inference(question, tokens=250):
    # Ensure model components are initially on CPU
    model.model.decoder.embed_tokens.to('cpu')
    model.model.decoder.embed_positions.to('cpu')
    for layer in model.model.decoder.layers:
        layer.to('cpu')
    model.lm_head.to('cpu')
    
    max_position_embeddings = model.config.max_position_embeddings
    print(f"Model's max_position_embeddings: {max_position_embeddings}")

    # Initialize input_ids_for_next_step. This will be the tensor that grows.
    input_ids_for_next_step = tok(question, return_tensors='pt')['input_ids'] 
    
    gc.collect()
    torch.cuda.empty_cache()

    for _ in range(tokens):
        # The 'x' variable for the current iteration is the input_ids_for_next_step
        current_input_ids = input_ids_for_next_step 
        current_seq_len = current_input_ids.shape[1]

        # --- Truncation ---
        if current_seq_len > max_position_embeddings:
            current_input_ids = current_input_ids[:, current_seq_len - max_position_embeddings:]
            print(f"Warning: Sequence truncated from {current_seq_len} to {current_input_ids.shape[1]} for max_position_embeddings.")
            current_seq_len = current_input_ids.shape[1] # Update after truncation

        # --- Embeddings ---
        model.model.decoder.embed_tokens.to('cpu')
        # Use current_input_ids for embedding lookup
        x_embeddings = model.model.decoder.embed_tokens(current_input_ids.to('cpu')) # x_embeddings is on CPU
        model.model.decoder.embed_tokens.to('cpu') # Move layer back
        gc.collect()
        
        # 2. Add Positional Embeddings
        model.model.decoder.embed_positions.to('cpu')
        try:
            position_offset = model.model.decoder.embed_positions.offset
        except AttributeError:
            position_offset = 2 # Common default for OPT

        pos_ids = torch.arange(position_offset, position_offset + current_seq_len, device='cpu').unsqueeze(0) 
        
        # The main `x` variable throughout the layer processing will be `current_token_embeddings_plus_pos`
        current_token_embeddings_plus_pos = x_embeddings + model.model.decoder.embed_positions(pos_ids) 
        model.model.decoder.embed_positions.to('cpu') 
        del x_embeddings 
        del pos_ids 
        gc.collect()

        # --- Decoder Layers Loop ---
        # Rename 'x' to something that reflects its current state (embeddings in, embeddings out)
        x_current_layer_output = current_token_embeddings_plus_pos # Start of layer processing
        for j, layer in enumerate(model.model.decoder.layers):
            try:
                layer.to('cpu')
                x_current_layer_output = x_current_layer_output.to('cpu') 

                with torch.no_grad():
                    x_current_layer_output = layer(x_current_layer_output)[0]

                layer.to('cpu')
                x_current_layer_output = x_current_layer_output.to('cpu') 
                
                gc.collect()

            except Exception as e:
                print(f"Error in decoder layer {j}: {e}")
                break
        else: 
            # --- LM Head and Token Generation ---
            model.lm_head.to('cpu')
            # The input to lm_head is x_current_layer_output (the final embeddings from decoder)
            logits = model.lm_head(x_current_layer_output.to('cpu')) 
            model.lm_head.to('cpu') 
            
            logits = logits.to('cpu') # Ensure logits are on CPU
            gc.collect()

            # Get the predicted token ID (on CPU)
            # Take the last token's prediction from batch 0
            predicted_token_id = logits.argmax(-1)[:, -1].item() 

            decoded_token = tok.decode(predicted_token_id)
            print(decoded_token, end=" ")
            
            # --- Prepare input_ids_for_next_step for the next iteration ---
            # Append the new token ID to the input_ids_for_next_step tensor (on CPU)
            new_token_tensor = torch.tensor([[predicted_token_id]], dtype=torch.long, device='cpu')
            
            # This is the correct concatenation for input_ids
            input_ids_for_next_step = torch.cat((input_ids_for_next_step, new_token_tensor), dim=1)
            
            # Update the 'question' string for printing and future reference
            question = question + " " + decoded_token
            
        if 'predicted_token_id' not in locals(): # If inner loop broke, stop outer loop
            break

In [4]:
def model_inference(question,tokens=250):
    for i in range(tokens):
        prompt=tok(question,return_tensors='pt').to('cuda')
        input_ids=prompt['input_ids']
        del prompt
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cuda')
        x = model.model.decoder.embed_tokens(input_ids)
        del input_ids
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cpu')
        # del model.model.decoder.embed_tokens
        torch.cuda.empty_cache()
        model.model.decoder.embed_positions.to('cuda')
        x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
        model.model.decoder.embed_positions.to('cpu')
        # del model.model.decoder.embed_positions
        torch.cuda.empty_cache()
        for i in model.model.decoder.layers:
            try:
                i.to('cuda')
                x=x.to('cuda')
                with torch.no_grad():
                    x=i(x)[0]
                i.to('cpu')
                torch.cuda.empty_cache()
                x=x.to('cpu')
                torch.cuda.empty_cache()
                # time.sleep(3)
            except:
                print(x)
                break
        model.lm_head.to('cuda')
        x=model.lm_head(x.to('cuda'))
        model.lm_head.to('cpu')
        torch.cuda.empty_cache()
        x1=x.argmax(-1)
        print(tok.decode(x1[0]),end=" ")
        question=question+" "+tok.decode(x1[0])
                

In [22]:
model_inference("How are you")

.''. your retarded're .''. you retarded're .dylib mom retard.''. 

NameError: name 'e' is not defined

In [11]:
print()
cnt=0
for i in model.model.decoder.layers:
    cnt+=1
    print(cnt)
    try:
        i.to('cuda')
        x.to('cuda')
        with torch.no_grad():
            x=i(x)[0]
        i.to('cpu')
        torch.cuda.empty_cache()
        x.to('cpu')
        torch.cuda.empty_cache()
        # time.sleep(3)
    except:
        print(cnt)
        print(x)
        break


1
2
3
4
5
6
7
8
9
10
11
12


In [12]:
model.lm_head.to('cuda')
x=model.lm_head(x.to('cuda'))
model.lm_head.to('cpu')
torch.cuda.empty_cache()

In [13]:
x1=x.argmax(-1)

In [14]:
tok.decode(x1[0])

' hi'